In [148]:
from lane_changing import *
from data import build_params

def build_result_mat(results_dry, results_wet):
    p_dry = np.zeros([3])
    p_wet = np.zeros([3])

    p_dry[ results_dry['a_opt_0'] ] += results_dry['prob_sensor_00'] + results_dry['prob_sensor_10']
    p_dry[ results_dry['a_opt_1'] ] += results_dry['prob_sensor_10'] + results_dry['prob_sensor_11']

    p_wet[ results_dry['a_opt_0'] ] += results_wet['prob_sensor_00'] + results_wet['prob_sensor_10']
    p_wet[ results_dry['a_opt_1'] ] += results_wet['prob_sensor_10'] + results_wet['prob_sensor_11']

    p_mat = np.array([p_dry, p_wet])   
    
    prob_s_dry = np.round(results_dry['prob_s'], 3)
    #prob_s_dry = prob_s_dry/np.sum(prob_s_dry)
    prob_s_wet = np.round(results_wet['prob_s'], 3)
    #prob_s_wet = prob_s_dry/np.sum(prob_s_wet)
    prob_s_mat = np.array([prob_s_dry, prob_s_wet])
    
    return p_mat, prob_s_mat

# Section 3.2.2

For the reference experiments we choose the following parameterization:

* ADS weights: `wA  = np.array([3/100, 22/100, 0, 3/100, 22/100, 0, 22/100, 31/100])`
* ADS risk aversion coefficient: `rho_A=0.5`

* MV weights: `np.array([0.1, 0.5, 0.05, 0.35])`
* MV risk aversion coefficient: `rho_MV=1.0`

* 1 passenger in ADS, 1 passenger in MV, 1 pedestrian.

Notice that this is a quite risky driving scene, in the sense that both the
ADS and the MV give a lot of weight to speed (last component of the vectors)
compared to the weight given to safety.


In [149]:
wA  = np.array([3/100, 22/100, 0, 3/100, 22/100, 0, 22/100, 31/100])
wMV = np.array([0.1, 0.5, 0.05, 0.35])

params = build_params(theta2=1.0, theta3=1.0, theta4=1.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)

In [150]:
results_wet = ads.simulate(theta=1)
results_dry = ads.simulate(theta=0)
p_mat, prob_s_mat = build_result_mat(results_dry, results_wet)

### Table 4

In [151]:
d = {
     'Pavement': ['Dry', 'Wet'],
     '$a_1$'   : p_mat[:,0],
     '$a_2$'   : p_mat[:,1],
     '$a_3$'   : p_mat[:,2],
    }
df = pd.DataFrame(d).set_index('Pavement')
df

,$a_1$,$a_2$,$a_3$
Pavement,,,
Dry,0.95,0.05,0.0
Wet,0.05,0.95,0.0


When pavement is dry, the ADS decides to change lane more often than not. 
When pavement is wet, it tends to be more conservative and decides more often not to change lane.

In [152]:
df.to_latex()

'\\begin{tabular}{lrrr}\n\\toprule\n{} &  \\$a\\_1\\$ &  \\$a\\_2\\$ &  \\$a\\_3\\$ \\\\\nPavement &        &        &        \\\\\n\\midrule\nDry      &   0.95 &   0.05 &    0.0 \\\\\nWet      &   0.05 &   0.95 &    0.0 \\\\\n\\bottomrule\n\\end{tabular}\n'

### Table 5

In [154]:
d = {
     'Pavement': ['Dry', 'Wet'],
     '$s_1$'   : prob_s_mat[:,0],
     '$s_2$'   : prob_s_mat[:,1],
     '$s_3$'   : prob_s_mat[:,2],
     '$s_4$'   : prob_s_mat[:,3]

    }
df = pd.DataFrame(d).set_index('Pavement')
df

,$s_1$,$s_2$,$s_3$,$s_4$
Pavement,,,,
Dry,0.158,0.158,0.683,0.0
Wet,0.017,0.017,0.967,0.0


This is a consequence of the actions chosen. When pavement is dry, ADS chooses to change lane more often and thus, risky scenarios ($s_1$ and $s_2$) occurr more often.

In [155]:
df.to_latex()

'\\begin{tabular}{lrrrr}\n\\toprule\n{} &  \\$s\\_1\\$ &  \\$s\\_2\\$ &  \\$s\\_3\\$ &  \\$s\\_4\\$ \\\\\nPavement &        &        &        &        \\\\\n\\midrule\nDry      &  0.158 &  0.158 &  0.683 &    0.0 \\\\\nWet      &  0.017 &  0.017 &  0.967 &    0.0 \\\\\n\\bottomrule\n\\end{tabular}\n'

### Table 6

Same weights as before. We compare 3 configurations.

* No passengers in ADS, 5 passengers in MV, 1 pedestrian.
* 4 in ADS, 1 passengers in MV, 1 pedestrian.
* 5 in ADS, 5 passengers in MV, 0 pedestrians.

In [156]:
wA  = np.array([3/100, 22/100, 0, 3/100, 22/100, 0, 22/100, 31/100])
wMV = np.array([0.1, 0.5, 0.05, 0.35])


In [157]:
params = build_params(theta2=0.0, theta3=5.0, theta4=1.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)

results_wet_1 = ads.simulate(theta=1)
results_dry_1 = ads.simulate(theta=0)

p_mat_1, prob_s_mat_1 = build_result_mat(results_dry_1, results_wet_1)

In [158]:
params = build_params(theta2=4.0, theta3=1.0, theta4=1.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)

results_wet_2 = ads.simulate(theta=1)
results_dry_2 = ads.simulate(theta=0)

p_mat_2, prob_s_mat_2 = build_result_mat(results_dry_2, results_wet_2)

In [159]:
params = build_params(theta2=5.0, theta3=5.0, theta4=0.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)

results_wet_3 = ads.simulate(theta=1)
results_dry_3 = ads.simulate(theta=0)

p_mat_3, prob_s_mat_3 = build_result_mat(results_dry_3, results_wet_3)

In [166]:
tuples =  [('1', 'Dry'),('1', 'Wet'),('2', 'Dry'),('3', 'Wet'),('3', 'Dry'),('3', 'Wet')]
index = pd.MultiIndex.from_tuples(tuples, names=["Configuration", "Pavement"])

conf1 = np.concatenate([p_mat_1, prob_s_mat_1], axis=1)
conf2 = np.concatenate([p_mat_2, prob_s_mat_2], axis=1)
conf3 = np.concatenate([p_mat_3, prob_s_mat_3], axis=1)
tab6  = pd.DataFrame( np.concatenate([conf1, conf2, conf3], axis=0) )
tab6.columns = ['$a_1$', '$a_2$', '$a_3$', '$s_1$', '$s_2$', '$s_3$', '$s_4$']
tab6.index   = index
tab6

$a_1$  $a_2$  $a_3$  $s_1$  $s_2$  $s_3$  $s_4$
Configuration Pavement                                                 
1             Dry        0.95   0.05    0.0  0.040  0.040  0.921    0.0
              Wet        0.05   0.95    0.0  0.017  0.017  0.967    0.0
2             Dry        0.00   1.00    0.0  0.000  0.000  1.000    0.0
3             Wet        0.00   1.00    0.0  0.000  0.000  1.000    0.0
              Dry        0.00   1.00    0.0  0.000  0.000  1.000    0.0
              Wet        0.00   1.00    0.0  0.000  0.000  1.000    0.0

In configuration 1-Dry, as in the previous case,
the ADS decides to change lane 95% of the times.
However, in this case, the probability of risky scenarios $s_1$ and $s_2$ is much lower.
This is a consequence of the fact that, as the MV has more passengers, it tends to be more autoprotective, and instead of deciding to accelerate most of the time, it decides to change lane.

In [169]:
tab6.to_latex()

'\\begin{tabular}{llrrrrrrr}\n\\toprule\n  &     &  \\$a\\_1\\$ &  \\$a\\_2\\$ &  \\$a\\_3\\$ &  \\$s\\_1\\$ &  \\$s\\_2\\$ &  \\$s\\_3\\$ &  \\$s\\_4\\$ \\\\\nConfiguration & Pavement &        &        &        &        &        &        &        \\\\\n\\midrule\n1 & Dry &   0.95 &   0.05 &    0.0 &  0.040 &  0.040 &  0.921 &    0.0 \\\\\n  & Wet &   0.05 &   0.95 &    0.0 &  0.017 &  0.017 &  0.967 &    0.0 \\\\\n2 & Dry &   0.00 &   1.00 &    0.0 &  0.000 &  0.000 &  1.000 &    0.0 \\\\\n3 & Wet &   0.00 &   1.00 &    0.0 &  0.000 &  0.000 &  1.000 &    0.0 \\\\\n  & Dry &   0.00 &   1.00 &    0.0 &  0.000 &  0.000 &  1.000 &    0.0 \\\\\n  & Wet &   0.00 &   1.00 &    0.0 &  0.000 &  0.000 &  1.000 &    0.0 \\\\\n\\bottomrule\n\\end{tabular}\n'

## Impact of Number of Pedestrians

In [95]:
wA  = np.array([5/100, 32/100, 0, 5/100, 32/100, 0, 22/100, 7/100])
wMV = np.array([0.05, 0.1, 0.05, 0.8])

params = build_params(theta2=5.0, theta3=5.0, theta4=2.0, 
                      rho_A=0.5, rho_MV=1.0,
                      weights_A=wA, weights_MV=wMV)
ads = lane_changing(params)
results_wet = ads.simulate(theta=1)
results_dry = ads.simulate(theta=0)

In [96]:
results_dry

{'prob_sensor_00': 0.9025,
 'prob_sensor_01': 0.04750000000000004,
 'prob_sensor_10': 0.04750000000000004,
 'prob_sensor_11': 0.0025000000000000044,
 'a_opt_0': 1,
 'a_opt_1': 1,
 'm_opt_00': 0,
 'm_opt_01': 0,
 'm_opt_10': 0,
 'm_opt_11': 0,
 'prob_s': array([0., 0., 1., 0.])}

## Impact of Risk aversion Coefficient